In [3]:
import keras
import os,sys
import numpy as np
import pandas as pd
import random
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization, Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
import PIL.Image as Image
import matplotlib.pyplot as plt
from tqdm import tqdm
Image.MAX_IMAGE_PIXELS = None

In [22]:
Image.

In [23]:
def resize_img(input_dir):
    im = Image.open(input_dir)
    (x, y) = im.size  # 读取图片尺寸（像素）
    x_1 = x//scale  # 定义缩小后的标准宽度
    y_1 = int(y * x_1 / x)  # 计算缩小后的高度
    out = im.resize((x_1, y_1))  # 改变尺寸，保持图片高品质
    #判断图片的通道模式，若图片在RGBA模式下，需先将其转变为RGB模式
    if out.mode=='RGBA':
        #转化为rgb格式
        out=out.convert('RGB')
        #最后保存为jpg格式的图片，这里因为图片本身为jpg所以后缀不更改
    return out

def save_mini_img():
    for i_ in range(start_page, end_page, step):
        i_ = str(i_) if i_ >= 10 else '0' + str(i_)
        for k_ in ['test','train']:
            j_li = ['a','b'] if k_ == 'test' else train_abc
            for j_ in j_li:
                _PREFIX = f'/kaggle/input/vesuvius-challenge-ink-detection/{k_}/{j_}/surface_volume/'
                out = resize_img(_PREFIX + f"{i_}.tif")
                save_dir = f'{k_}/{j_}/'
                if not os.path.exists(save_dir):
                    os.makedirs(save_dir)
                out.save(save_dir + f'{k_}_{j_}_{i_}.tif')
                
def save_mini_img_2():
    for k_ in ['test','train']:
        print('k_',k_)
        j_li = ['a','b'] if k_ == 'test' else train_abc
        for j_ in j_li:
            _PREFIX = f'/kaggle/input/vesuvius-challenge-ink-detection/{k_}/{j_}/'
            out = resize_img(_PREFIX + f"mask.png")
            save_dir = f'{k_}/{j_}/'
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            out.save(save_dir + f'{k_}_{j_}_mask.png')
        
        if k_ == 'train':
            for j_ in j_li:
                _PREFIX = f'/kaggle/input/vesuvius-challenge-ink-detection/{k_}/{j_}/'
                print(_PREFIX + f"inklabels.png")
                out = resize_img(_PREFIX + f"inklabels.png")
                save_dir = f'{k_}/{j_}/'
                if not os.path.exists(save_dir):
                    os.makedirs(save_dir)
                out.save(save_dir + f'{k_}_{j_}_inklabels.png')

In [13]:
# save_mini_img()
# save_mini_img_2()
# base_dir = '/kaggle/working/'
base_dir = 'dataset/'

In [5]:
from keras import layers

def VGG16(img_input):
    # Block 1
    x = layers.Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv1')(img_input)
    x = layers.Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv2')(x)
    feat1 = x
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = layers.Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv1')(x)
    x = layers.Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv2')(x)
    feat2 = x
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)


    # Block 3
    x = layers.Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv1')(x)
    x = layers.Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv2')(x)
    x = layers.Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv3')(x)
    feat3 = x
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv1')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv2')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv3')(x)
    feat4 = x

    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv1')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv2')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv3')(x)
    feat5 = x
    return feat1, feat2, feat3, feat4, feat5


In [2]:
import numpy as np
from keras.models import *
from keras.layers import *


def Unet(input_shape=(256,256,3), num_classes=21):
    inputs = Input(input_shape)
    feat1, feat2, feat3, feat4, feat5 = VGG16(inputs) 
      
    channels = [64, 128, 256, 512]

    P5_up = UpSampling2D(size=(2, 2))(feat5)
    P4 = Concatenate(axis=3)([feat4, P5_up])
    P4 = Conv2D(channels[3], 3, activation='relu', padding='same', kernel_initializer='he_normal')(P4)
    P4 = Conv2D(channels[3], 3, activation='relu', padding='same', kernel_initializer='he_normal')(P4)

    P4_up = UpSampling2D(size=(2, 2))(P4)
    P3 = Concatenate(axis=3)([feat3, P4_up])
    P3 = Conv2D(channels[2], 3, activation='relu', padding='same', kernel_initializer='he_normal')(P3)
    P3 = Conv2D(channels[2], 3, activation='relu', padding='same', kernel_initializer='he_normal')(P3)

    P3_up = UpSampling2D(size=(2, 2))(P3)
    P2 = Concatenate(axis=3)([feat2, P3_up])
    P2 = Conv2D(channels[1], 3, activation='relu', padding='same', kernel_initializer='he_normal')(P2)
    P2 = Conv2D(channels[1], 3, activation='relu', padding='same', kernel_initializer='he_normal')(P2)

    P2_up = UpSampling2D(size=(2, 2))(P2)
    P1 = Concatenate(axis=3)([feat1, P2_up])
    P1 = Conv2D(channels[0], 3, activation='relu', padding='same', kernel_initializer='he_normal')(P1)
    P1 = Conv2D(channels[0], 3, activation='relu', padding='same', kernel_initializer='he_normal')(P1)

    P1 = Conv2D(num_classes, 1, activation="softmax")(P1)

    model = Model(inputs=inputs, outputs=P1)
    return model


In [ ]:
subm = ''
for c_ in ['a','b']:
    _tmp = pre_img_to_submission(pre_imgs[c_])
    subm = subm + f'{c_},' + _tmp + '\n'
subm = 'Id,Predicted\n' + subm

In [ ]:
print(subm.strip(), file=open('submission.csv', 'w'))

In [6]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [9]:
size = 224
in_chans = 6
train_aug_list = [
        # A.RandomResizedCrop(
        #     size, size, scale=(0.85, 1.0)),
        A.Resize(size, size),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.75),
        A.ShiftScaleRotate(p=0.75),
        A.OneOf([
                A.GaussNoise(var_limit=[10, 50]),
                A.GaussianBlur(),
                A.MotionBlur(),
                ], p=0.4),
        A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
        A.CoarseDropout(max_holes=1, max_width=int(size * 0.3), max_height=int(size * 0.3), 
                        mask_fill_value=0, p=0.5),
        # A.Cutout(max_h_size=int(size * 0.6),
        #          max_w_size=int(size * 0.6), num_holes=1, p=1.0),
        A.Normalize(
            mean= [0] * in_chans,
            std= [1] * in_chans
        ),
        ToTensorV2(transpose_mask=True),
    ]

valid_aug_list = [
    A.Resize(size, size),
    A.Normalize(
        mean= [0] * in_chans,
        std= [1] * in_chans
    ),
    ToTensorV2(transpose_mask=True),
]

In [18]:
transform = A.Compose(train_aug_list)

In [19]:
image = base_dir+'train/1/train_1_27.tif'
img_trans = transform(image=np.array(image))  # image为图像地址'xxx.jpg'

NameError: name 'np' is not defined